<div style="text-align: justify">

# 4.9 A senha de tamanho variável

Nomes: Andressa Cristine M. Costa e Andriel Vinicius Martins  da Silva

**Objetivo:** Resolver o problema da senha de forma que você não forneça a informação
do tamanho da senha para a função que gera a população. Considere que a senha pode
ser uma string de 1 até 30 caracteres.

**Dica:** A função objetivo terá que quantificar em sua métrica tanto se o candidato
acertou as letras quanto se acertou o tamanho da senha.

**Dica 2:** Você pode criar diferentes estratégias de mutação, não precisa ser apensa
uma! Quem sabe uma função de mutação pode alterar letras e a outra pode alterar o
tamanho da senha? Ver o exercício “Praticamente um X-man!”.

**Dica 3:** Observe que você terá que pensar um pouco sobre como fará o cruzamento no
caso de senhas de tamanhos diferentes. Quem sabe tenha que fazer alguma consideração
adicional sobre quais são os valores possíveis para o ponto de corte…

## Contextualização

&nbsp;&nbsp;&nbsp;&nbsp;Nesse problema queremos adivinhar uma senha utilizando algoritmos genéticos de forma que não precisamos revelar o tamanho da senha de entrada para o algoritmo. Para isso, foram implementadas pequenas variações ao código apresentado em aula.

&nbsp;&nbsp;&nbsp;&nbsp;No código original, a estratégia de resolução consistia em "chutar" aleatóriamente letras que compunham um tamanho especificado de senha e medir a distância entre o código do caractere do chute e o código do caractere da senha correta, calculando a distância entre eles. Nesse caso, precisamos fornecer "chutes" tanto para acertar os caracteres, quanto para acertar o tamanho da senha inserida. Por isso, foi necessário modificar levemente a lógica do código original.



<div style="text-align: justify">

##  Resolução
Modificações feitas em [1].

Precisamos atualizar certas funções do arquivo "funcoes_senha_var.py" para atender as necessidades específicas desse problema.

**I) Modificar a forma como o candidato é criado:** No caso, queremos que isso seja feito de forma independente do tamanho da senha.
Na célula abaixo, modificamos a lógica de gerar um candidato para sorteio para receber um valor inteiro aleatório para representar o tamanho da senha de chute inicial. Após isso, geramos uma senha com esse tamanho por meio da função `gene_senha(letras_possiveis)`.

```python
import random
def cria_candidato_senha(letras_possiveis): 
    """Cria um candidato para o problema da senha

    Args:
      indice_tamanho: guarda o tamanho sorteado para o chute de senha
      letras_possiveis: letras possíveis de serem sorteadas.

    """
    candidato = []
    indice_tamanho = random.randint(1, 30)

    for _ in range(indice_tamanho):
        candidato.append(gene_senha(letras_possiveis))

    return candidato

<div style="text-align: justify">

**II) Modificação da função objetivo:** precisamos retirar a parte referente à variável que informava o tamanho da senha e inserir um condicional para verificar se o tamanho da senha de palpite é válido. Apenas se o tamanho do palpite for igual ao da senha verdadeira a comparação entre as distâncias dos caracteres será realizada.

```python
def funcao_objetivo_senha(candidato, senha_verdadeira):
    """Computa a funcao objetivo de um candidato no problema da senha

    Args:
      candidato: um palpite para a senha que você quer descobrir
      senha_verdadeira: a senha que você está tentando descobrir

    """
    distancia = 0

    if len(candidato) == len(senha_verdadeira):
        for letra_candidato, letra_senha in zip(candidato, senha_verdadeira):
            num_letra_candidato = ord(letra_candidato)
            num_letra_senha = ord(letra_senha)
            distancia += abs(num_letra_candidato - num_letra_senha)

        return distancia    
    else:
        return float("inf")

Retornamos o `float("inf")` nesse ponto para criar um filtro de seleção para os indivíduos. 

**III) Execução do código de algoritmos genéticos:** Nessa parte, empregamos o código conforme fornecido em aula.

In [1]:
from string import ascii_lowercase, ascii_uppercase, digits

from funcoes_senha_var import populacao_senha as cria_populacao
from funcoes_senha_var import funcao_objetivo_pop_senha as funcao_objetivo
from funcoes_senha_var import selecao_torneio_min as funcao_selecao
from funcoes_senha_var import cruzamento_uniforme as funcao_cruzamento
from funcoes_senha_var import mutacao_simples as funcao_mutacao1
from funcoes_senha_var import mutacao_salto as funcao_mutacao2

In [2]:
SENHA = list("59laranjasDeliciosas")
CARACTERES_POSSIVEIS = ascii_lowercase + ascii_uppercase + digits

TAMANHO_POPULACAO = 100
CHANCE_DE_CRUZAMENTO = 0.5
CHANCE_DE_MUTACAO = 0.025
TAMANHO_TORNEIO = 10

In [3]:
populacao = cria_populacao(TAMANHO_POPULACAO, CARACTERES_POSSIVEIS)

In [4]:
menor_fitness_geral = float("inf")
geracao = 0

while menor_fitness_geral != 0:
    
    # Seleção
    fitness = funcao_objetivo(populacao, SENHA)        
    selecionados = funcao_selecao(populacao, fitness, TAMANHO_TORNEIO)
    
    # Cruzamento
    proxima_geracao = []
    for pai, mae in zip(selecionados[::2], selecionados[1::2]):
        individuo1, individuo2 = funcao_cruzamento(pai, mae, CHANCE_DE_CRUZAMENTO)
        proxima_geracao.append(individuo1)
        proxima_geracao.append(individuo2)
    
    # Mutação
    funcao_mutacao1(proxima_geracao, CHANCE_DE_MUTACAO, list(CARACTERES_POSSIVEIS))
    funcao_mutacao2(proxima_geracao, CHANCE_DE_MUTACAO, list(CARACTERES_POSSIVEIS))
    
    # Encerramento
    populacao = proxima_geracao
    geracao += 1
    
    fitness = funcao_objetivo(populacao, SENHA)
    menor_fitness_observado = min(fitness)
    
    if menor_fitness_observado < menor_fitness_geral:
        menor_fitness_geral = menor_fitness_observado
        indice = fitness.index(menor_fitness_observado)
        candidato = populacao[indice]
        print(geracao, "".join(candidato))

1 H7PmiHortgd8zzmLYfUY
2 H7ACLHorgg5tzzmvYqsY
3 27XTiHoSFjMtzzfvVfUY
4 H7PTiHorgjMtzzfvYfsY
5 H7gmiHorgjI8zimjYxUY
6 27gTiHorgjItzimvYxUY
7 27gTiHorgjItzifjYqUY
8 27gTiHorgjEtzifjYqUY
9 27gTiHoogjItzifjYqeY
10 27gTiHoogjEtzifjYqeY
11 27gmiHoogjEtzifjYqeY
12 27gTiHojgjEtzifjYqeY
13 27gTixojgjEtzifjYqeY
14 28gTixojgjEtzifjYqeY
16 28gmixojgjEtzifjYqeY
17 28gmixojgjEszifjYqeY
18 28gmixojgjEsziejYqeY
19 28gmiRojgjEszifjYqeY
20 28gmiRojgjEsziejYqeY
21 28gmjRojgjEsziejYqeY
22 28gmjRojgjEsnifjYqeY
23 28gmjRojfjEsnifjYqeY
24 28gmjRojfjEsniejYqeY
25 29gljRojfjEsniejYqeY
28 29gljRojfjEsniejZqeY
29 29gkjRojfjEsniejZqeY
30 29gkjRojejEsniejZqeY
32 29gkjVojfjEsniejZqeY
33 29gkjVnjfjEsniejZqeY
34 29gkjVnjfjEsmiejZqeY
36 29gkjVnjekEsniejZqeY
38 29gkjVnjdkEsniejZqeY
39 29gkjVnjdjEbniejZreY
41 29gkjVnjdjEbniejyred
43 29gkjanjdjEbniejyred
45 29gkjanjdjEbniejyree
46 29gkqanjdjEbniejyred
47 29gkqanjdjEbniejyree
48 29ghqanjdjEbniejyree
49 29ggqanjdjEbniejyree
50 29gfqanjdjEbniejyree
51 29gfranjdjEbniejyree
5

### Conclusão

<div style="text-align: justify">

&nbsp;&nbsp;&nbsp;&nbsp;Ao definir infinito para a distância retornada para os comprimentos diferentes da senha alvo, teremos uma espécie de filtro durante a seleção dos indivíduos, uma vez que a seleção funciona por torneio mínimo. Isso permite uma rápida convergência para o comprimento esperado. Após convergir, o código irá funcionar exatamente como no problema da senha com comprimento único. Provalvemente esse não será o resultado mais otimizado, porém cumpre o objetivo.


### Referências

[1].CASSAR, Daniel. [Material de sala de aula]. Redes Neurais e Algoritmos Genéticos, 2025, Ilum - Escola de Ciência.